# Create a Script from a Jupyter Notebook
Sometime it is useful to transform your notebook in an actual script e.g.:

- if you want it to be executed automatically (unattended)
- if you want to create a module out of it in order to use it in a bigger application 

## extract code from Jupyter
Jupyter has a very wide range of formats to export the content of a notebook; some of them are a graphical export (e.g. html or pdf -- this requires a latex installation) or textual export (ascii, rtf), etc. 

To start creating a python script from your notebook, open it into Jupyter, then
1. from the File menu select `save and export notebook as`
2. from the submenu select `Executable script`
this will save a file in your Download directory named with the title of your notebook, but with .py extension

You can now open this file with your favorite code editor; here is what you will see:

- before every cell there will be a comment with the cell execution number
- every code cell will be copied in an individual block of commands
- markdown cells and raw cells will be presented as comments
- output will be removed

In the following sections I will list some cleanup actions which will help you transform this script into a more manageable piece of code

## manage magic code
magic code is translated into the equivalent python call e.g.

```
?sum
```

becomes

```python
get_ipython().run_line_magic('pinfo', 'sum')
```

most of the time you may want to get rid of all of this kind of code as some functionalities (e.g. accessing documentation) are intended only for interactive usage within a jupyter notebook.

Other functionalities (e.g. timing your cell execution) may be better managed with other libraries.

## add code to save tables
Jupyter conveniently shows pandas `DataFrame`s as tables; you may want to extract these results into files.

To access these tables from a script you can save them into files

If your table are small you may be willing to save them in some simple format:
- csv using
  ```python
  df.to_csv("my_file.csv")
  ```
- [Microsoft Excel](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html) using `openpyxl` [optional library](https://openpyxl.readthedocs.io/en/stable/tutorial.html) 
  ```python
  df.to_excel("my_file.xlsx")
  ```

For larger tables or more complex tasks binary formats may help.
- [Apache Parquet](https://parquet.apache.org/) is a columnar binary format ideal for large data collections and high performance computation; it requires some optional library e.g. [pyarrow](https://arrow.apache.org/docs/python/index.html) see [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_parquet.html)
  ```python
  df.to_parquet("my_file.pqt")
  ```
- [hdf5](https://www.hdfgroup.org/solutions/hdf5/) is a binary format which can contain multiple tables in a single file (see [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_hdf.html) )
    ```python
  df.to_hdf("my_file.pqt")
  ```

## add code to save figures

## clean up the code
The following suggestions holds for any python script and are not strictly required for the execution.

- move all `import` statements at the beginning of the file
- organize the code in functions and classes; possibly add type annotations
- create a single entry point at the bottom of the code with the usual
  ```python
  if __name__ == "__main__":
      main()
    ```
- add command line options management using libraries like `optparse` [(see here)](https://docs.python.org/3/library/optparse.html)
- separate data and configuration from code: libraries like `toml` [(see here)](https://github.com/uiri/toml) can help reading configuration files 
- transform absolute paths into relative paths
- consider using pyproject.html to [collect dependencies and constraints](https://packaging.python.org/en/latest/guides/writing-pyproject-toml/)
- consider using a linter to evaluate code inconsistencies
- create unit tests to verify your functions individually; `pytest` helps in this [task](https://docs.pytest.org/en/stable/)
- add documentation per each function or class as well as a module doc string

## add shell script to launch the code